In [1]:
import disentangler as dt
from disentangler.dependencies import *

In [2]:
rna_model = mira.topics.load_model(dt.bonemarrow.config.MIRA_RNA_NO_CORRECTION)
atac_model = mira.topics.load_model(dt.bonemarrow.config.MIRA_ATAC_NO_CORRECTION)

INFO:mira.topic_model.base:Moving model to CPU for inference.
INFO:mira.topic_model.base:Moving model to device: cpu
/liulab/alynch/projects/batcheffect-env/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/liulab/alynch/projects/batcheffect-env/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
INFO:mira.topic_model.base:Moving model to CPU for inference.
INFO:mira.topic_model.base:Moving model to device: cpu


In [3]:
rna_data, atac_data = dt.bonemarrow.utils.load_GEX_data(), dt.bonemarrow.utils.load_ATAC_data()

In [4]:
atac_model.predict(atac_data)
rna_model.predict(rna_data)

Predicting latent vars:   0%|          | 0/263 [00:00<?, ?it/s]

INFO:mira.adata_interface.core:Added key to obsm: X_topic_compositions
INFO:mira.adata_interface.topic_model:Added cols: topic_0, topic_1, topic_2, topic_3, topic_4, topic_5, topic_6, topic_7, topic_8, topic_9, topic_10, topic_11, topic_12, topic_13, topic_14, topic_15, topic_16, topic_17, topic_18, topic_19, topic_20, topic_21, topic_22, topic_23, topic_24, topic_25, topic_26, topic_27, topic_28, topic_29, topic_30
INFO:mira.adata_interface.core:Added key to varm: topic_feature_compositions
INFO:mira.adata_interface.core:Added key to varm: topic_feature_activations


Predicting latent vars:   0%|          | 0/247 [00:00<?, ?it/s]

INFO:mira.adata_interface.core:Added key to obsm: X_topic_compositions
INFO:mira.adata_interface.topic_model:Added cols: topic_0, topic_1, topic_2, topic_3, topic_4, topic_5, topic_6, topic_7, topic_8, topic_9, topic_10, topic_11, topic_12, topic_13, topic_14, topic_15, topic_16, topic_17, topic_18, topic_19, topic_20, topic_21, topic_22, topic_23, topic_24, topic_25
INFO:mira.adata_interface.core:Added key to varm: topic_feature_compositions
INFO:mira.adata_interface.core:Added key to varm: topic_feature_activations


In [5]:
atac_model.get_umap_features(atac_data, box_cox=0.2)
rna_model.get_umap_features(rna_data, box_cox=0.2)

INFO:mira.adata_interface.topic_model:Fetching key X_topic_compositions from obsm
INFO:mira.adata_interface.core:Added key to obsm: X_umap_features
INFO:mira.adata_interface.topic_model:Fetching key X_topic_compositions from obsm
INFO:mira.adata_interface.core:Added key to obsm: X_umap_features


In [6]:
rna_data, atac_data = mira.utils.make_joint_representation(rna_data, atac_data)

INFO:mira.adata_interface.utils:61127 out of 69076 cells shared between datasets (88%).
INFO:mira.adata_interface.utils:Key added to obsm: X_joint_umap_features


In [7]:
rp_args = dict(
    expr_adata = rna_data,
    atac_adata = atac_data,
)

In [8]:
atac_data.var[['chr','start','end']] = \
    list(map(list, atac_data.var_names.str.split('-')))

In [9]:
mira.tl.get_distance_to_TSS(atac_data,
                            tss_data = 'data/bonemarrow/hg38_tss_data.bed12.filtered',
                            genome_file= 'data/bonemarrow/hg38.chrom.sizes'
                           )

INFO:mira.tools.connect_genes_peaks:Finding peak intersections with promoters ...
INFO:mira.tools.connect_genes_peaks:Calculating distances between peaks and TSS ...
INFO:mira.tools.connect_genes_peaks:Masking other genes' promoters ...
INFO:mira.adata_interface.rp_model:Added key to var: distance_to_TSS
INFO:mira.adata_interface.rp_model:Added key to uns: distance_to_TSS_genes


In [14]:
litemodel = mira.rp.LITE_Model(
    expr_model=rna_model,
    accessibility_model=atac_model,
    genes=np.intersect1d(rna_model.features, atac_data.uns['distance_to_TSS_genes'])[1951:],
    counts_layer='counts',
)

In [15]:
!mkdir -p data/bonemarrow/rp/litemodels_NOBATCH

In [16]:
litemodel.fit(**rp_args, callback=mira.rp.SaveCallback('data/bonemarrow/rp/litemodels_NOBATCH/'))

Fitting models:   0%|          | 0/891 [00:00<?, ?it/s]